# Reflection


In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions.
This is then used downstream for things like re-planning, search, or evaluation.

![Reflection](./img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.

#### Prerequisites

We will be using a basic agent with a search tool here.

In [1]:
# %pip install -U --quiet  langchain langgraph
# %pip install -U --quiet tavily-python

## Generate

For our example, we will create a "5 paragraph essay" generator. First, create the generator:


In [2]:
from langchain_community.chat_models import GigaChat
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы помощник по написанию докладов, ваша задача — писать отличные доклад из 5 абзацев."
            " Создайте лучший доклад по запросу пользователя."
            " Если пользователь предоставляет критику, ответьте переработанной версией ваших предыдущих попыток.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = GigaChat(credentials="...")
generate = prompt | llm

In [20]:
essay = ""
request = HumanMessage(
    content="Значение 'Маленького принца' в детстве современных детей"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

"Маленький принц" - это замечательная книга Антуана де Сент-Экзюпери, которая была написана в 1943 году. Она рассказывает историю о маленьком принце, который путешествует по разным планетам, встречая разных персонажей и узнавая о жизни и ее ценностях.

В современном мире, где дети окружены технологиями и быстро меняющимися трендами, "Маленький принц" остается актуальным и важным произведением. Эта книга помогает детям понять, что важно в жизни, учит их ценить простые вещи и находить красоту в обыденных вещах.

Одной из главных тем книги является любовь и забота о других. Маленький принц учит детей, что забота о других и их благополучии - это важная часть жизни. Он также учит детей, что важно быть ответственным и заботиться о своих обязанностях.

Кроме того, "Маленький принц" помогает детям развивать свое воображение и творческое мышление. Книга полна метафор и символов, которые помогают детям понять сложные концепции и идеи.

В целом, "Маленький принц" остается важным произведением для

### Reflect

In [21]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы учитель, оценивающий представленый доклад. Сформируйте критику и рекомендации для представленной работы пользователя."
            " Предоставьте подробные рекомендации, включая требования к объему, глубине, стилю и т.д.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [22]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Ваш ответ абсолютно верен. "Маленький принц" действительно помогает детям понимать ценности жизни, учит их быть заботливыми и ответственными, а также развивает их воображение и творческое мышление. Ваше объяснение также очень точное и полное.

Однако, я бы рекомендовал добавить больше деталей и примеров из книги, чтобы более полно объяснить, как "Маленький принц" помогает детям развивать эти качества. Например, можно было бы упомянуть, что Маленький принц учит детей ценить простые вещи, такие как забота о розе на его планете, и что это помогает им находить красоту в обыденных вещах. Также можно было бы упомянуть, что книга полна метафор и символов, которые помогают детям понять сложные концепции и идеи, например, символ змеи, который символизирует смерть и возрождение.

Кроме того, можно было бы добавить, что "Маленький принц" помогает детям развивать эмоциональный интеллект, учит их эмпатии и пониманию других людей. Например, можно было бы упомянуть, что Маленький принц учит детей, чт

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [23]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Спасибо за ваш отзыв и рекомендации! Я учту их при написании следующего доклада. Вот переработанная версия моего предыдущего ответа:

"Маленький принц" - это не просто детская книга, это произведение, которое помогает детям понимать ценности жизни, учит их быть заботливыми и ответственными, а также развивает их воображение и творческое мышление.

Одной из главных тем книги является любовь и забота о других. Маленький принц учит детей, что забота о других и их благополучии - это важная часть жизни. Он также учит детей, что важно быть ответственным и заботиться о своих обязанностях.

Кроме того, "Маленький принц" помогает детям развивать свое воображение и творческое мышление. Книга полна метафор и символов, которые помогают детям понять сложные концепции и идеи. Например, символ змеи, который символизирует смерть и возрождение, помогает детям понять, что все в жизни имеет свой конец и начало.

"Маленький принц" также помогает детям развивать эмоциональный интеллект, учит их эмпатии и по

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.

In [24]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [25]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Значение 'Маленького принца' в детстве современных детей"
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content='"Маленький принц" - это книга, которая затрагивает важные темы, такие как любовь, дружба, ответственность и потеря. Она учит детей ценить простые вещи в жизни и не забывать о том, что действительно важно. В современном мире, где дети часто погружены в технологии и социальные сети, "Маленький принц" может стать источником вдохновения и напоминанием о том, что важно сохранять детскую невинность и любопытство. Книга помогает детям развивать свое воображение и учит их видеть красоту в простых вещах. Она также подчеркивает важность заботы о других и умения прощать. В целом, "Маленький принц" остается актуальным и важным произведением для современных детей, помогая им развивать свои ценности и умения.')}
---
{'reflect': HumanMessage(content='Ваше утверждение о том, что "Маленький принц" является важным произведением для современных детей, верно. Книга затрагивает важные темы, которые актуальны и в наше время. Она помогает детям развивать свое воображение, цени

In [26]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

================================ Human Message =================================

Значение 'Маленького принца' в детстве современных детей

================================== Ai Message ==================================

"Маленький принц" - это книга, которая затрагивает важные темы, такие как любовь, дружба, ответственность и потеря. Она учит детей ценить простые вещи в жизни и не забывать о том, что действительно важно. В современном мире, где дети часто погружены в технологии и социальные сети, "Маленький принц" может стать источником вдохновения и напоминанием о том, что важно сохранять детскую невинность и любопытство. Книга помогает детям развивать свое воображение и учит их видеть красоту в простых вещах. Она также подчеркивает важность заботы о других и умения прощать. В целом, "Маленький принц" остается актуальным и важным произведением для современных детей, помогая им развивать свои ценности и умения.

================================ Human Message =========================

## Conclusion

Now that you've applied reflection to an LLM agent, I'll note one thing: self-reflection is inherantly cyclic: it is much more effective if the reflection step has additional context or feedback (from tool observations, checks, etc.). If, like in the scenario above, the reflection step simply prompts the LLM to reflect on its output, it can still benefit the output quality (since the LLM then has multiple "shots" at getting a good output), but it's less guaranteed.
